In [10]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

### web scraping (escolher condição e nº de genes)

In [11]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    #print (a)

import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
#print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
else:
    print (n_genes)

    


escolha o que quer pesquisar: gli2
escolha o nº de genes que quer obter (máximo 20): 20
['2736', '14633', '2736', '304729', '395956', '510255', '100520334', '100682984', '459582', '102447306', '100757572', '100067420', '125696933', '123378859', '122688694', '122453941', '121072061', '120374651', '116975547', '116507904']


## Tabela "Gene"

### Buscar ids do ncbi:

In [12]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

X65482.1
A00829.1
X65482.1
M97702.1
X68316.1
AAADS0003348.1
AAADW0009410.1
ABAAM0346030.1
AAADX0043451.1
AAADK0043071.1
DS196524.1
EC923692.1
AAVX01203288.1
DH405380.1
BAAF03073813.1


### Buscar Links dos genes:

In [13]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(Ids)

['https://www.ncbi.nlm.nih.gov/nuccore/X65482.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/A00829.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/X65482.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/M97702.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/X68316.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADS0003348.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADW0009410.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/ABAAM0346030.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADX0043451.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADK0043071.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/DS196524.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/EC923692.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAVX01203288.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/DH405380.1',
 'https://www.ncbi.nlm.nih.gov/nuccore/BAAF03073813.1']

### Buscar organismos: (novo)

In [ ]:
#organismos

Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
print(Organismos)

### Buscar seq dos genes:

In [ ]:
#Biopython
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

### Buscar % de Alanina:

In [ ]:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= (count/len(c)*100)
    Adenina.append(perc)
print (Adenina)

### Buscar % de Citosina:

In [ ]:
Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= (count/len(c)*100)
    Citosina.append(perc)
print (Citosina)

### Buscar % de Guanina:

In [ ]:
Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= (count/len(c)*100)
    Guanina.append(perc)
print (Guanina)

### Buscar % de Timina:

In [ ]:
Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= (count/len(c)*100)
    Timina.append(perc)
print (Timina)

### Buscar gene description:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

### Buscar Data:

In [ ]:
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])
print(data)

### Buscar tamanho:

In [ ]:

database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))

print(tam)

## Tabela "CDS"

### Buscar quantidade de cds e a sua localização:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
pos=[]
for info in records:
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            cds= i
            a= str(cds)
            b= len(a)
            print (b)
            featcds.append(b)
            #position=i
            #pos.append(info.features[position].location)
            #print (pos)
    #for h in info.features:
        #if h.type["CDS"] == none:
            #featcds.append("sem resultados")
print(teste)     
            

## Tabela "Uniprot"

### Buscar Ids de proteinas:

In [ ]:
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
            
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

### Buscar Links de proteinas:

In [ ]:
import time
ID_PROT
def url_get_id_p(a):
    url_list_p=[ ]
    for id_p in ID_PROT:
        url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(id_p)
        url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(ID_PROT)
print(url_ids_protein)  # but we don't need to save this links

### Buscar id proteina (uniprot):

In [ ]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [ ]:
#final version
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function", "sequence"]

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)


result = {}
for field in fields:
    result[field] = [get_field_for_id(i, field) for i in ID_PROT]

for field, values in result.items():
#     print(f"Results for {field}")
#     print(values)
    #regex
    for x in values:
        entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
        organism = re.search(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL)
        function = re.search( r'b\'Function \[(CC)\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
        location = re.search( r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
        sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
        if entry:
            e = re.match(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(e.group(1))
        elif organism:
            o = re.match(r'b\'Organism\\n(.+?(?=\\n\'))', x, re.DOTALL )
            print(o.group(1))
        elif function:
            f = re.match(r'b\'Function .{15} (.+?(?=\\n\'))', x, re.DOTALL )
            print(f.group(1))
        elif location:
            l = re.match(r'b\'Subcellular location .{27} (.+?(?=\\n\'))', x, re.DOTALL )
            print(l.group(1))
        elif sequence:
            s = re.match(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            print(s.group(1) , len(s.group(1)) )
#         else:
#             print("fail")
#ADD CONDITION FOR EMPY RESULT (MEANS THE ID PROTEIN IS NOT REVIEWED)

## Tabela "Pubmed"

### Buscar id de artigos:

In [14]:
import requests
content_id = []
for url in url_get_id(Ids):
    r_id = requests.get(url)
    content_id.append(r_id.content)
    
import requests
from bs4 import BeautifulSoup
listas=[]
# Parsing the HTML
for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        #print(line)
        #if 'name' in line.attrs:
        #    print(line.attrs['name'])
        if 'content' in line.attrs:
            # print(line.attrs['content'])		
            id = line.attrs['content']

    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

    r2 = requests.get(url)
    r4= r2.content.decode('utf-8')
    #r4= str(r2.content)
    listas.append(r4)
cc= ', '.join(listas)
er= cc.replace('//','')
final= er.split(', ')
#print (er)

output_dict = {}
for x in final:
    version = re.search(r'VERSION\s+(.*?)\s', x)
    pubmed = re.search(r'PUBMED\s+(.*?)\s', x)
    if version:
#         print(version.group(1))
        versionf=version.group(1)
        output_dict.setdefault(version.group(1), [])
    if pubmed:
#         print(pubmed.group(1))
        output_dict[versionf].append(pubmed.group(1))
#     else:
       # output_dict[version.group(1)].append("N/A")

print(output_dict)



{'X65482.1': ['1525873', '1525873'], 'A00829.1': [], 'M97702.1': ['7683659'], 'X68316.1': ['8085790'], 'AAADS0003348.1': ['14663149', '14960301', '16141072', '16141073', '16645617'], 'AAADW0009410.1': ['14663149', '14960301', '16141072', '16141073', '16645617'], 'ABAAM0346030.1': ['14663149', '14960301', '16141072', '16141073', '16645617'], 'AAADX0043451.1': ['14663149', '14960301', '16141072', '16141073', '16645617'], 'AAADK0043071.1': ['14663149', '14960301', '16141072', '16141073', '16645617'], 'DS196524.1': [], 'EC923692.1': ['17761391'], 'AAVX01203288.1': ['17185593', '17407382'], 'DH405380.1': [], 'BAAF03073813.1': []}


In [16]:
keys_list = []
values_list = []
for key, vals in output_dict.items():
    if vals:
        for val in vals:
            keys_list.append(key)
            values_list.append(val)
    else:
        keys_list.append(key)
        values_list.append("N/A")

In [135]:
print(keys_list)

['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'AAADR0035516.1', 'AAADR0035516.1', 'AAADR0035516.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1', 'AAADU0045050.1', 'AAADU0045050.1', 'AAADU0045050.1', 'AAADU0045050.1']


In [149]:
print(values_list)

['3012462', 'N/A', '14663149', '14960301', '16141072', '16141073', '16645617', '8116255', '14663149', '14960301', '16141072', '16141073', '16645617']


In [17]:
new_list_ = []
seen = set()
for item in values_list:
    if item not in seen:
        seen.add(item)
        new_list_.append(item)
print(new_list_)

['1525873', 'N/A', '7683659', '8085790', '14663149', '14960301', '16141072', '16141073', '16645617', '17761391', '17185593', '17407382']


In [148]:
print(B)

{'3012462', '14663149', '16645617', '8116255', 'N/A', '16141072', '16141073', '14960301'}


In [9]:
number_map = {}
next_number = 1
output = []
for number in values_list:
    if number not in number_map:
        number_map[number] = next_number
        next_number += 1
    output.append(number_map[number])

print(output)

[1, 2, 3, 4, 5, 6, 7, 8, 3, 4, 5, 6, 7]


In [137]:
so_ids_pubmed = [val for key, vals in output_dict.items() for val in vals]
so_ids_ncbi = list(output_dict.keys())

In [144]:
A= set(so_ids_pubmed)

In [145]:
print (A)

{'3012462', '14663149', '16645617', '8116255', '16141073', '16141072', '14960301'}


In [247]:
print(so_ids_ncbi)

['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1']


In [240]:
#lista_final=[]
#for x in final:
    #Pub = re.search(r'VERSION\s+.*?(?=\s)|(PUBMED\s+.*?(?=\s))', x, re.DOTALL)
    #if Pub:
        #print (Pub.group(0))
        #lista_final.append(Pub.group(0))
   
    
#print(lista_final)
#limpar= "' ".join(lista_final)
#id_pubmedES= limpar.split('VERSION     ')     ######## criar dicionário aqui!!!!
#sem_virgulas= ', '.join(id_pubmedES)
#m= sem_virgulas[2:-1]
#limpo= m.replace('PUBMED   ','')
#limpinho= limpo.replace("'" ,'')
#id_pu= limpinho.split(' , ')
#print(id_pu)

#lista_final_final=[]
#for x in id_pu:
    #if x.count(' ') == 0:
        #rep= x

#replace=",".join(id_pu)
#NA= replace.replace(f'{rep}','RESULTADO: N/A')
#id_FINAL= NA.split(',')
#print(id_FINAL)

#for h in id_FINAL:
    #FINA = re.search(r'\s.*?(?=$)', h, re.DOTALL)
    #if FINA:
        #print (FINA.group(0))
        #lista_final_final.append([FINA.group(0)])
    #else:
        #lista_final_final.append([])

#for i in lista_final_final:
    #print (i)
#SE=",".join(lista_final_final[0])
#id_FI= SE.split(' ')
#print(lista_final_final)


In [ ]:
#FINAL
#result_dict = {}
#database = 'nucleotide'
#email= 'rodrigoce9@gmail.com'
#idlist=Ids
# idlist= Ids.remove('U49845')
#for ids in idlist:
    #list_pro=[]
    #handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
   #records = list(SeqIO.parse(handle,"gb"))
    #handle.close()
    #for info in records:
        #list_pro.append(info.id)
        #for i in info.features:
            #if i.type == "CDS":
                #pro= str(i.qualifiers["protein_id"])
                #list_pro.append(pro)
        #if len(list_pro)==1: #if no protein_id was found
            #result_dict[info.id] = "N/A"
        #else:
            #list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            #result_dict[info.id] = ', '.join(list_pro[1:])
#print(result_dict)
#key: id_genebank; values: id_Uniprot

### Buscar artigos:

In [18]:
titles= []
authors=[]
source=[]
affiliation=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= new_list_
counter = 0
for i in idlist:
    if i!= "N/A":
        handle = Entrez.efetch(db=database, id=i, rettype="medline", retmode="text") 
        records = Medline.parse(handle)
        for info in records:
            print(f'id: {i}')
            print("title:", info.get("TI", "-"))
            titles.append(info.get("TI", "-"))
            print('authors: ', info.get("AU", "-"))
            authors_string = info.get("AU", "-")
            if len(authors_string) > 5:
                authors_h = authors_string[0:5]
                authors.append(authors_h)
            else:
                authors.append(authors_string) 
            print("source:", info.get("SO", "-"))
            #source.append(info.get(("SO", ["N/A"]))
            print("affiliation: ", info.get("AD", ["N/A"]) ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
            affiliation_string= info.get("AD", ["N/A"])
            if len(affiliation_string) > 5:
                affiliation_h = affiliation_string[0:5]
                affiliation.append(affiliation_h)
            else:
                affiliation.append(affiliation_string)
            print()
            counter += 1
    else:
        print(f'id: {i} - N/A')
        titles.append("N/A")
        authors.append(["N/A"])
        source.append("N/A")
        affiliation.append(["N/A"])
        #print()
        counter += 1


id: 1525873
title: Ribosomal DNA internal transcribed spacers are highly divergent in the phytopathogenic ascomycete Fusarium sambucinum (Gibberella pulicaris).
authors:  ["O'Donnell K"]
source: Curr Genet. 1992 Sep;22(3):213-20. doi: 10.1007/BF00351728.
affiliation:  ['National Center for Agricultural Utilization Research, Agricultural Research Service, Peoria, IL 61604.']

id: N/A - N/A
id: 7683659
title: The glutathione S-transferase D genes. A divergently organized, intronless gene family in Drosophila melanogaster.
authors:  ['Toung YP', 'Hsieh TS', 'Tu CP']
source: J Biol Chem. 1993 May 5;268(13):9737-46.
affiliation:  ['Department of Molecular and Cell Biology, Pennsylvania State University, University Park 16802.']

id: 8085790
title: Genetic interrelationships of proteolytic Clostridium botulinum types A, B, and F and other members of the Clostridium botulinum complex as revealed by small-subunit rRNA gene sequences.
authors:  ['Hutson RA', 'Thompson DE', 'Lawson PA', 'Schocke

### buscar titles

In [310]:
titles = [ [title] for title in titles]

### buscar doi

In [ ]:
doi_list = []
for x in source:
    match = re.search("doi: (.*)", x)
    if match:
        doi_list.append(match.group(1))
    else:
        doi_list.append("N/A")

### buscar authors:

In [11]:
authors_new = []
for sub_list in authors:
    for author in sub_list:
        authors_new.append([author])


In [12]:
id_authors_dict = {i: authors[counter] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

In [13]:
print(id_authors_dict)

{'3012462': ['Leutwiler LS', 'Meyerowitz EM', 'Tobin EM'], 'N/A': ['N/A'], '14663149': ['Shiraki T', 'Kondo S', 'Katayama S', 'Waki K', 'Kasukawa T'], '14960301': ['Kodzius R', 'Matsumura Y', 'Kasukawa T', 'Shimokawa K', 'Fukuda S'], '16141072': ['Carninci P', 'Kasukawa T', 'Katayama S', 'Gough J', 'Frith MC'], '16141073': ['Katayama S', 'Tomaru Y', 'Kasukawa T', 'Waki K', 'Nakanishi M'], '16645617': ['Carninci P', 'Sandelin A', 'Lenhard B', 'Katayama S', 'Shimokawa K'], '8116255': ['Song KJ', 'Nerurkar VR', 'Saitou N', 'Lazo A', 'Blakeslee JR']}


In [14]:
pubmed_list = []
authors_list = []
for key, vals in id_authors_dict.items():
    if vals:
        for val in vals:
            pubmed_list.append(key)
            authors_list.append(val)
    #else:
        #pubmed_list.append(key)
        #authors_list .append("N/A")

In [15]:
print(pubmed_list)

['3012462', '3012462', '3012462', 'N/A', '14663149', '14663149', '14663149', '14663149', '14663149', '14960301', '14960301', '14960301', '14960301', '14960301', '16141072', '16141072', '16141072', '16141072', '16141072', '16141073', '16141073', '16141073', '16141073', '16141073', '16645617', '16645617', '16645617', '16645617', '16645617', '8116255', '8116255', '8116255', '8116255', '8116255']


In [16]:
print(authors_list)

['Leutwiler LS', 'Meyerowitz EM', 'Tobin EM', 'N/A', 'Shiraki T', 'Kondo S', 'Katayama S', 'Waki K', 'Kasukawa T', 'Kodzius R', 'Matsumura Y', 'Kasukawa T', 'Shimokawa K', 'Fukuda S', 'Carninci P', 'Kasukawa T', 'Katayama S', 'Gough J', 'Frith MC', 'Katayama S', 'Tomaru Y', 'Kasukawa T', 'Waki K', 'Nakanishi M', 'Carninci P', 'Sandelin A', 'Lenhard B', 'Katayama S', 'Shimokawa K', 'Song KJ', 'Nerurkar VR', 'Saitou N', 'Lazo A', 'Blakeslee JR']


In [17]:
new_list_authors= []
seen = set()
for item in authors_list:
    if item not in seen:
        seen.add(item)
        new_list_authors.append(item)
print(new_list_authors)

['Leutwiler LS', 'Meyerowitz EM', 'Tobin EM', 'N/A', 'Shiraki T', 'Kondo S', 'Katayama S', 'Waki K', 'Kasukawa T', 'Kodzius R', 'Matsumura Y', 'Shimokawa K', 'Fukuda S', 'Carninci P', 'Gough J', 'Frith MC', 'Tomaru Y', 'Nakanishi M', 'Sandelin A', 'Lenhard B', 'Song KJ', 'Nerurkar VR', 'Saitou N', 'Lazo A', 'Blakeslee JR']


In [ ]:
number_map_pub = {}
next_number = 1
output_pub = []
for number in pubmed_list:
    if number not in number_map_pub:
        number_map_pub[number] = next_number
        next_number += 1
    output_pub.append(number_map_pub[number])

In [18]:
number_map_ = {}
next_number = 1
output_authors = []
for number in authors_list:
    if number not in number_map:
        number_map_[number] = next_number
        next_number += 1
    output.append(number_map[number])

print(output)

[1, 2, 3, 4, 5, 6, 7, 8, 3, 4, 5, 6, 7, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 9, 12, 13, 14, 9, 7, 15, 16, 7, 17, 9, 8, 18, 14, 19, 20, 7, 12, 21, 22, 23, 24, 25]


### buscar affil:

In [19]:
print(affiliation)

[['N/A'], ['N/A'], ['Laboratory for Genome Exploration Research Group, RIKEN Genomic Sciences Center, Yokohama Institute 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, Kanagawa 230-0045, Japan.'], ['Genome Science Laboratory, RIKEN, Wako Main Campus, Hirosawa 2-1, Wako, Saitama 351-0198, Japan.'], ['N/A'], ['Laboratory for Genome Exploration Research Group, RIKEN Genomic Sciences Centre (GSC), RIKEN Yokohama Institute, 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, 230-0045, Japan.'], ['Genome Exploration Research Group, RIKEN Genomic Sciences Center, RIKEN Yokohama Institute, 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, Kanagawa, 230-0045, Japan.'], ['Laboratory of Central Nervous System Studies, National Institute of Neurological Disorders and Stroke, National Institutes of Health, Bethesda, Maryland 20892.']]


In [20]:
single_affiliation_list = []
for i in affiliation:
    single_affiliation_list.extend(i)

print(single_affiliation_list)

['N/A', 'N/A', 'Laboratory for Genome Exploration Research Group, RIKEN Genomic Sciences Center, Yokohama Institute 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, Kanagawa 230-0045, Japan.', 'Genome Science Laboratory, RIKEN, Wako Main Campus, Hirosawa 2-1, Wako, Saitama 351-0198, Japan.', 'N/A', 'Laboratory for Genome Exploration Research Group, RIKEN Genomic Sciences Centre (GSC), RIKEN Yokohama Institute, 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, 230-0045, Japan.', 'Genome Exploration Research Group, RIKEN Genomic Sciences Center, RIKEN Yokohama Institute, 1-7-22 Suehiro-cho, Tsurumi-ku, Yokohama, Kanagawa, 230-0045, Japan.', 'Laboratory of Central Nervous System Studies, National Institute of Neurological Disorders and Stroke, National Institutes of Health, Bethesda, Maryland 20892.']


In [22]:
affiliation_new = []
for sub_list in affiliation:
    for affiliation in sub_list:
        affiliation_new.append([affiliation])
#print(affiliation_new)

id_affiliation_dict = {i: [single_affiliation_list[counter]] if i != "N/A" else ["N/A"] for counter, i in enumerate(idlist)}

pubmed_affi_list = []
affi_list = []
for key, vals in id_affiliation_dict.items():
    if vals:
        for val in vals:
            pubmed_affi_list.append(key)
            affi_list.append(val)
    else:
        pubmed_affi_list.append(key)
        affi_list.append(["N/A"])

In [ ]:
new_list_affi= []
seen = set()
for item in single_affiliation_list :
    if item not in seen:
        seen.add(item)
        new_list_affi.append(item)

number_map_affi_pub = {}
next_number = int(SEARCH_ID_[0])
output_pub_affi = []
for number in pubmed_affi_listt:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_pub_affi.append(number_map_affi[number])
    
number_map_affi = {}
next_number = 1
output_affi = []
for number in affi_list:
    if number not in number_map_affi:
        number_map_affi[number] = next_number
        next_number += 1
    output_affi.append(number_map_affi[number])
